In [1]:
import os 


In [2]:
%pwd

'd:\\ML projects\\Chicken-Disease-Classification-\\research'

In [3]:
#change the directory to the root folder 
os.chdir("../")

In [4]:
%pwd

'd:\\ML projects\\Chicken-Disease-Classification-'

In [5]:
#entity folder conatins code for what should be the return type of the function 

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path 

In [6]:
%pwd

'd:\\ML projects\\Chicken-Disease-Classification-'

In [7]:
from src.cnnClassifier.constants import *

In [8]:
!pip install -e .


Obtaining file:///D:/ML%20projects/Chicken-Disease-Classification-
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: cnnClassifier
    Found existing installation: cnnClassifier 0.0.0
    Uninstalling cnnClassifier-0.0.0:
      Successfully uninstalled cnnClassifier-0.0.0
  Running setup.py develop for cnnClassifier


  DEPRECATION: Legacy editable install of cnnClassifier==0.0.0 from file:///D:/ML%20projects/Chicken-Disease-Classification- (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [11]:

import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-11 13:23:03,216: INFO:'common': yaml file: config\config.yaml loaded successfully]
[2025-01-11 13:23:03,242: INFO:'common': yaml file: params.yaml loaded successfully]
[2025-01-11 13:23:03,249: INFO:'common': created directory at: artifacts]
[2025-01-11 13:23:03,257: INFO:'common': created directory at: artifacts/data_ingestion]
[2025-01-11 13:23:26,251: INFO:'4037744573': artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 522A:2E8082:2E4FA8:3B04C0:67822367
Accept-Ranges: bytes
Date: Sat, 11 Jan 2025 07:53:14 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4729-BOM
X-Cac